In [1]:
data_root = "../../data/"
#stats stuff
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

# ML stuff
import numpy as np
from numpy.fft import *
import torch
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso
import pandas as pd
import lightgbm as lgb

# DL stuff
from torch.autograd import Variable
from fastprogress import master_bar, progress_bar
import torch
import torch.nn as nn
from torch.utils.data import Dataset


# plotting
import matplotlib.pyplot as plt
import seaborn as sns


# basic stuff
import datetime
import requests
import io
from collections import Counter


# relative imports
from LSTM_preprocess import get_Xy, get_train_test, get_train_df, sliding_windows_mutli_features
from hfuncs import check_mkdir

In [2]:
#################### LOAD DATA ######################
nasdaq = pd.read_csv(data_root + "NASDAQ_100_Data_From_2010.csv", sep="\t")

window_size = 30
train_ratio = 0.80
device = torch.device('cpu')
features = ['Adj_Close', 'log_Volatility', 'log_Volume', 'log_Adj_Close']
using = ['FB', 'TSLA', 'AAPL', 'AMZN', 'NVDA', 'MSFT', 'GOOGL']
# AAPL(Apple), MSFT(Microsoft), GOOGL(Google), AMZN(Amazon), TSLA(Tesla), FB(Facebook), NVDA(Nvidia)

In [4]:
df, features = get_train_df(nasdaq, using, features)
x, y = get_Xy(df, window_size)
print('x.shape, y.shape',x.shape, y.shape)
trainX, trainY, testX, testY = get_train_test(x, y, train_ratio)

28 28
x.shape, y.shape (3372, 30, 22) (3372, 7)


In [ ]:
class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        #self.seq_length = seq_length

        self.dropout = nn.Dropout(p=0.2)
        
        # what does the batch_first do
        self.lstm = nn.LSTM(\
            input_size=input_size, 
            hidden_size=hidden_size,
            num_layers=num_layers, 
            batch_first=True,
            dropout = 0.25)
        
        # Linear(in_features, out_features)
        self.fc = nn.Linear(hidden_size, num_classes)                                                                                                                                                                                                                           

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size).to(device))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)                                         
        out = self.dropout(out)
       
        return out

# create a nn class (just-for-fun choice :-) 
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))





In [ ]:
###### Parameters #######
num_epochs = 500
learning_rate = 1e-3
input_size = 22 # features(?)
hidden_size = 512
num_layers = 1
num_classes = 7 # because we are using 7 stocks
#########################
best_val_loss = 100                        

In [ ]:
######################################## ONLY RUN FOR TRAINING ######################################
### Init Model
lstm = LSTM(num_classes, input_size, hidden_size, num_layers)
lstm.to(device)

### where to save models
check_mkdir("LSTM_3_out")

### Set Criterion Optimizer and scheduler
criterion = torch.nn.MSELoss().to(device) 
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate, weight_decay=1e-5)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=500, factor=0.5, min_lr=1e-7, eps=1e-08)

#optimizer = torch.optim.SGD(lstm.parameters(), lr=learning_rate)

# Train model
for epoch in progress_bar(range(num_epochs)):
    lstm.train()
    outputs= lstm(trainX.to(device))
    optimizer.zero_grad()

    # obtain loss func
    loss = criterion(outputs, trainY.to(device))
    loss.backward()

    optimizer.step()

    #evaluate on test
    lstm.eval()
    valid = lstm(testX.to(device))
    vall_loss = criterion(valid, testY.to(device))

    scheduler.step(vall_loss)

    if vall_loss.cpu().item() < best_val_loss:
         torch.save(lstm.state_dict(), 'LSTM_3/best_model.pt')
         print("saved best model epoch:",epoch,"val loss is:",vall_loss.cpu().item())
         best_val_loss = vall_loss.cpu().item()

    if epoch%50==0:
        print(f"Epoch: {epoch}, loss: {loss.cpu().item()}, valid loss:{vall_loss.cpu().item()}")


# check_mkdir("LSTM_3_out")
# torch.save(lstm.state_dict(), 'LSTM_3_out/model1_1.pt')